In [1]:
import pandas as pd
import numpy as np
import os
import librosa
%matplotlib inline

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split

import pickle

path = input('path to file: ')

if path == '':
    path = 'A30003X6.wav'

pd.set_option('display.max_rows', 200)
#path = 'A30003X6.wav'
sr = 8000
n_fft = 2048
n_mels = 128
fmax = 4000

path to file: 


In [2]:
# создаём столбцы под обогащение данных
df = pd.DataFrame()
df['Spectral_centroid'] = np.nan
df['Rolloff'] = np.nan
df['Bandwidth'] = np.nan
df['Max_mean_freq_1'] = np.nan
df['Max_mean_freq_2'] = np.nan
df['Max_mean_freq_3'] = np.nan
df['Max_mean_freq_all_1'] = np.nan
df['Max_mean_freq_all_2'] = np.nan
df['Max_mean_freq_all_3'] = np.nan
df['Freq_min_all'] = np.nan
df['Freq_min'] = np.nan

In [3]:
# выгружаем данные по регрессии
A  = open('coeffs.pkl', 'rb')
coeffs, intercepts, classes = pickle.load(A)
A.close()
clf_done = LogisticRegression()
clf_done.coef_ = coeffs
clf_done.intercept_ = intercepts
clf_done.classes_ = classes

# выгружаем данные по нормировке
B = open('scaler.pkl', 'rb')
scale, mean, n_samples, var = pickle.load(B)
B.close()
std_sc = StandardScaler()
std_sc.scale_ = scale
std_sc.mean_ = mean
std_sc.n_samples_seen_ = n_samples
std_sc.var_ = var


In [4]:
# определим функцию, вырезающую фрагмент с голосом (начало и конец определяются условием амплитуды выше средней по файлу)
def get_position (x, sr, coeff=5):
    z = np.array([np.abs(i) for i in x])
    k = 0
    l = len(z) - 1
    while z[k] < coeff * z.mean():
        k += 1
    while (z[l] < 7 * coeff * z.mean()) and (l > k + 1000):
        l -= 1
    bottom = round(k/sr,2)
    top = round(l/sr,2)
    return bottom, top

def get_position_alt (x, sr, coeff=5, dur = 0.5):
    z = np.array([np.abs(i) for i in x])
    k = 0
    while z[k] < coeff * z.mean():
        k += 1
    bottom = round(k/sr,2)
    top = bottom + dur
    return bottom, top

In [5]:
# выкачиваем и преобразуем звуковые фрагменты
x, sr_out = librosa.load(path, sr=sr, mono=True)

S, phase = librosa.magphase(librosa.feature.melspectrogram(y=x, sr=sr, n_mels=n_mels, fmax=fmax))
# средний спектр по записи
D = S.mean(axis=1)
D[0] = 0
d = list(D)

# добавляем частоту с максимальным средним в спектре 1
df.loc[0,'Max_mean_freq_all_1'] = d.index(D.max()) * sr / n_mels

D[d.index(D.max())] = 0
D[d.index(D.max())-1] = 0
if d.index(D.max()) < n_mels:
    D[d.index(D.max())+1] = 0
d = list(D)
# добавляем частоту с максимальным средним в спектре 2
df.loc[0,'Max_mean_freq_all_2'] = d.index(D.max()) * sr / n_mels

D[d.index(D.max())] = 0
D[d.index(D.max())-1] = 0
if d.index(D.max()) < n_mels:
    D[d.index(D.max())+1] = 0
d = list(D)
# добавляем частоту с максимальным средним в спектре 3
df.loc[0,'Max_mean_freq_all_3'] = d.index(D.max()) * sr / n_mels
# средний спектр по записи
D = S.mean(axis=1)
D[0] = 0
d = list(D)
ind = 0
while D[ind] < D.mean():
    ind += 1
# добавдяем минимальную частоту, где проявляется значение выше среднего
df.loc[0,'Freq_min_all'] = ind * sr / n_mels

while D[ind] < D.mean() * 1.5:
    ind += 1
# добавдяем минимальную частоту, где проявляется значение выше 1.5 среднего
df.loc[0,'Freq_min_all_1'] = ind * sr / n_mels

while D[ind] < D.mean() * 2:
    ind += 1
# добавдяем минимальную частоту, где проявляется значение выше 2 средних
df.loc[0,'Freq_min_all_2'] = ind * sr / n_mels

ind = len(D) - 1

while D[ind] < D.mean():
    ind -= 1
# добавдяем максимальную частоту, где проявляется значение выше среднего
df.loc[0,'Freq_max_all'] = ind * sr / n_mels

while D[ind] < D.mean() * 1.5:
    ind -= 1
# добавдяем максимальную частоту, где проявляется значение выше 1.5 среднего
df.loc[0,'Freq_max_all_1'] = ind * sr / n_mels

while D[ind] < D.mean() * 2:
    ind -= 1
# добавдяем максимальную частоту, где проявляется значение выше 2 средних
df.loc[0,'Freq_max_all_2'] = ind * sr / n_mels

# делаем тоже самое для отрезка с голосом
bottom, top = get_position(x, sr, coeff=5)
x, sr_out = librosa.load(path, sr=sr, mono=True,
                    offset=bottom, duration=top-bottom)

S, phase = librosa.magphase(librosa.feature.melspectrogram(y=x, sr=sr, n_mels=n_mels, fmax=fmax))
D = S.mean(axis=1)
D[0] = 0

d = list(D)

# добавляем частоту с максимальным средним в спектре 1
df.loc[0,'Max_mean_freq_1'] = d.index(D.max()) * sr / n_mels

D[d.index(D.max())] = 0
D[d.index(D.max())-1] = 0
if d.index(D.max()) < n_mels:
    D[d.index(D.max())+1] = 0
d = list(D)
# добавляем частоту с максимальным средним в спектре 2
df.loc[0,'Max_mean_freq_2'] = d.index(D.max()) * sr / n_mels

D[d.index(D.max())] = 0
D[d.index(D.max())-1] = 0
if d.index(D.max()) < n_mels:
    D[d.index(D.max())+1] = 0
d = list(D)
# добавляем частоту с максимальным средним в спектре 3
df.loc[0,'Max_mean_freq_3'] = d.index(D.max()) * sr / n_mels

D = S.mean(axis=1)
D[0] = 0
d = list(D)
ind = 0
while D[ind] < D.mean():
    ind += 1
# добавдяем минимальную частоту, где проявляется значение выше среднего
df.loc[0,'Freq_min'] = ind * sr / n_mels

while D[ind] < D.mean() * 1.5:
    ind += 1
# добавдяем минимальную частоту, где проявляется значение выше 1.5 среднего
df.loc[0,'Freq_min_1'] = ind * sr / n_mels

while D[ind] < D.mean() * 2:
    ind += 1
# добавдяем минимальную частоту, где проявляется значение выше 2 средних
df.loc[0,'Freq_min_2'] = ind * sr / n_mels


ind = len(D) - 1

while D[ind] < D.mean():
    ind -= 1
# добавдяем максимальную частоту, где проявляется значение выше среднего
df.loc[0,'Freq_max'] = ind * sr / n_mels


# добавляем усреднённый спектральный центроид
df.loc[0,'Spectral_centroid'] = librosa.feature.spectral_centroid(S=S)[0].mean()
# добавляем усреднённую спектральную ширину
df.loc[0,'Bandwidth'] = librosa.feature.spectral_bandwidth(S=S)[0].mean()
# спектральный спад
df.loc[0,'Rolloff'] = librosa.feature.spectral_rolloff(S=S, sr=sr)[0].mean()
# Скорость пересечения нуля


X = df[:].drop(['Max_mean_freq_2', 'Max_mean_freq_3', 'Max_mean_freq_all_2',
               'Max_mean_freq_all_3','Freq_max_all', 'Freq_max_all_1', ],1)


In [6]:
# нормализуем
#scale_features_std = StandardScaler() 
X = std_sc.transform(X) 

In [7]:
print('belongs to ', clf_done.predict([X[-1]])[0], ' ', round(max(clf_done.predict_proba(X)[-1]) * 100,1), '%')

belongs to  m   97.8 %
